#### Importing Libraries

In [1]:
from datetime import datetime
from datetime import date
from datetime import timedelta
import json
from bs4 import BeautifulSoup
import requests
from dataclasses import dataclass

#### Making the Holiday class

In [2]:
class Holiday:

    def __init__(self, name, date):
            
        date_format = '%Y-%m-%d'
        date = datetime.strptime(date, date_format)
        self._name = name 
        self._date = date

    def __str__(self):
        return f'The holiday is {self._name} and the date is {self._date}'

In [6]:
class HolidayList:
    
    innerHolidays = []
    
    def __init__(self, holidayObj):
        try:
            self.holidayObj = holidayObj
            print("Your holiday has been added!")
        except: 
            print("Something is wrong here. It's possible that your object is not a Holiday object.")

    def addHoliday(self, holidayObj):
        
        self.innerHolidays.append(vars(holidayObj))
        return self.innerHolidays

    def __str__(self):
        return f'The Holidays are as follows: {self.innerHolidays}'
    
    def __repr__(self):
        return f'the holiday is {self.holidayObj}, holiday list: {self.innerHolidays}'
    
    def findHoliday(self, HolidayName, Date):
        for item in self.innerHolidays:
            if item['_name'] == HolidayName:
                if item['_date'] == datetime.datetime.strptime(Date, '%Y-%m-%d'):
                    return f'The Holiday is {HolidayName} and the date is {Date}'
                else: 
                    return 'That holiday is not here; the date may be wrong.'
    
    def removeHoliday(self, HolidayName, Date):
        
        holiday_name = HolidayName
        holiday_date = Date
        holiday_date = datetime.strptime(holiday_date, '%Y-%m-%d')
        
        item_1 = [x for x in self.innerHolidays if x['_name'] == holiday_name]
        item_2 = [x for x in item_1 if x['_date'] == holiday_date]
        self.innerHolidays = list(filter(lambda x: x!= item_2[0], self.innerHolidays))

    def displayHolidaysandWeek(self):
        holiday_and_week_list =  list(map(lambda x:(x['_name'], x['_date'].isocalendar()[1]), self.innerHolidays))
        for holiday in holiday_and_week_list:
            print(f'{holiday[0]} occurs in week {holiday[1]} of the year')

    def read_json(self, filelocation):
        filelocation = open(filelocation)
        json_file = json.load(filelocation)
        for x in json_file['holidays']:
            self.innerHolidays.append(x)
    
    def scrapeHolidays(self):
        
        self.holidays_in_2020 = requests.get('https://www.timeanddate.com/holidays/us/2020').text
        self.holidays_in_2020_bs = BeautifulSoup(self.holidays_in_2020, 'html.parser')
        
        self.holidays_in_2021 = requests.get('https://www.timeanddate.com/holidays/us/2021').text
        self.holidays_in_2021_bs = BeautifulSoup(self.holidays_in_2021, 'html.parser')
        
        self.holidays_in_2022 = requests.get('https://www.timeanddate.com/holidays/us/2022').text
        self.holidays_in_2022_bs = BeautifulSoup(self.holidays_in_2022, 'html.parser')

        self.holidays_in_2023 = requests.get('https://www.timeanddate.com/holidays/us/2023').text
        self.holidays_in_2023_bs = BeautifulSoup(self.holidays_in_2023, 'html.parser')

        self.holidays_in_2024 = requests.get('https://www.timeanddate.com/holidays/us/2024').text
        self.holidays_in_2024_bs = BeautifulSoup(self.holidays_in_2024, 'html.parser')

        self.holiday_year_list = [self.holidays_in_2020_bs, self.holidays_in_2021_bs, self.holidays_in_2022_bs, self.holidays_in_2023_bs, self.holidays_in_2024_bs]
        
        holiday_list = []

        for self.holiday_list_item in self.holiday_year_list:

            holiday_list_year = self.holiday_list_item.title.contents[0][-4:]
            tbody = self.holiday_list_item.tbody
            trs = tbody.contents
            for tr in trs[1:]:
                if len(str(tr)) > 35:
                    holiday_dictionary = {}
                    holiday_dictionary['Holiday Name'] = tr.find('a').string
                    holiday_dictionary['Holiday Date'] = tr.find('th').string
                    holiday_dictionary['Holiday Year'] = holiday_list_year
                    self.holiday_fixed_date = holiday_dictionary['Holiday Date'] + " " + holiday_dictionary['Holiday Year']
                    self.holiday_fixed_date = datetime.strptime(self.holiday_fixed_date, "%b %d %Y")
                    holiday_dictionary['Holiday Fixed Date'] = self.holiday_fixed_date
                    holiday_list.append(holiday_dictionary)
                    if ({"_name": holiday_dictionary['Holiday Name'], 
                    "_date": holiday_dictionary['Holiday Fixed Date']}) not in self.innerHolidays:
                        self.innerHolidays.append({"_name": holiday_dictionary['Holiday Name'], 
                        "_date": holiday_dictionary['Holiday Fixed Date']})
    
    def getWeather(self):

        self.current_day = datetime.today()
        self.current_day_iso = self.current_day.isocalendar()
        self.current_day_formatted = self.current_day.strftime('%Y-%m-%d')
        if int(self.current_day_iso[2]) == 1:
            self.day7(self)
        elif int(self.current_day_iso[2]) == 2:
            self.day6(self)
        elif int(self.current_day_iso[2]) == 3:
            self.day5(self)
        elif int(self.current_day_iso[2]) == 4:
            self.day4(self)
        elif int(self.current_day_iso[2]) == 5:
            self.day3(self)
        elif int(self.current_day_iso[2]) == 6:
            self.day2(self)
        elif int(self.current_day_iso[2]) == 1:
            self.day1(self)
        
        self.url_history = "https://weatherapi-com.p.rapidapi.com/history.json"

        self.headers_history = {
            "X-RapidAPI-Host": "weatherapi-com.p.rapidapi.com",
            "X-RapidAPI-Key": "a137553bf2msh51f1d9425f948dfp1278d4jsn2653822db306"
        }
        self.response_history = requests.request("GET", self.url_history, headers = self.headers_history, params=self.querystring_history)

        self.json_weather_history = json.loads(self.response_history.text)

        self.narrowed_json = self.json_weather_history

        print(self.narrowed_json)

        self.datelist = []

        for i in range(len(self.narrowed_json)): 
            if self.narrowed_json[i]['date'] not in self.datelist:
                self.datelist.append(self.narrowed_json[i]['date'])
                for x in self.innerHolidays:
                    if x['_date'] == datetime.strptime(self.narrowed_json[i]['date'], '%Y-%m-%d'):                    
                        print(f'{x["_name"]} - {self.narrowed_json[i]["date"]}', "-", self.narrowed_json[i]['day']['condition']['text'])
                    
        
    def day7(self):
        self.starting_iso = self.current_day_iso
        self.starting_date = self.current_day - timedelta(days = 7)
        self.starting_iso = self.starting_date.strftime('%Y-%m-%d')
        self.end_date = self.current_day - timedelta(days = 1)
        self.end_date_iso = self.end_date.strftime('%Y-%m-%d')
        self.starting_iso = self.starting_date.strftime('%Y-%m-%d')
        self.querystring_history = {"q":"San Diego","dt":str(self.starting_iso),"lang":"en", 'end_dt': str(self.end_date_iso)}
    
    def day6(self):
        self.starting_iso = self.current_day_iso
        self.starting_date = self.current_day - timedelta(days = 6)
        self.starting_iso = self.starting_date.strftime('%Y-%m-%d')
        self.end_date = self.current_day + timedelta(days = 1)
        self.end_date_iso = self.end_date.strftime('%Y-%m-%d')
        self.starting_iso = self.starting_date.strftime('%Y-%m-%d')
        self.querystring_history = {"q":"San Diego","dt":str(self.starting_iso),"lang":"en", 'end_dt': str(self.end_date_iso)}

    def day5(self):
        self.starting_iso = self.current_day_iso
        self.starting_date = self.current_day - timedelta(days = 5)
        self.starting_iso = self.starting_date.strftime('%Y-%m-%d')
        self.end_date = self.current_day + timedelta(days = 2)
        self.end_date_iso = self.end_date.strftime('%Y-%m-%d')
        self.starting_iso = self.starting_date.strftime('%Y-%m-%d')
        self.querystring_history = {"q":"San Diego","dt":str(self.starting_iso),"lang":"en", 'end_dt': str(self.end_date_iso)}
    
    def day4(self):
        self.starting_iso = self.current_day_iso
        self.starting_date = self.current_day - timedelta(days = 4)
        self.starting_iso = self.starting_date.strftime('%Y-%m-%d')
        self.end_date = self.current_day + timedelta(days = 3)
        self.end_date_iso = self.end_date.strftime('%Y-%m-%d')
        self.starting_iso = self.starting_date.strftime('%Y-%m-%d')
        self.querystring_history = {"q":"San Diego","dt":str(self.starting_iso),"lang":"en", 'end_dt': str(self.end_date_iso)}

    def day3(self):
        self.starting_iso = self.current_day_iso
        self.starting_date = self.current_day - timedelta(days = 3)
        self.starting_iso = self.starting_date.strftime('%Y-%m-%d')
        self.end_date = self.current_day + timedelta(days = 4)
        self.end_date_iso = self.end_date.strftime('%Y-%m-%d')
        self.starting_iso = self.starting_date.strftime('%Y-%m-%d')
        self.querystring_history = {"q":"San Diego","dt":str(self.starting_iso),"lang":"en", 'end_dt': str(self.end_date_iso)}

    def day2(self):
        self.starting_iso = self.current_day_iso
        self.starting_date = self.current_day - timedelta(days = 2)
        self.starting_iso = self.starting_date.strftime('%Y-%m-%d')
        self.end_date = self.current_day + timedelta(days = 5)
        self.end_date_iso = self.end_date.strftime('%Y-%m-%d')
        self.starting_iso = self.starting_date.strftime('%Y-%m-%d')
        self.querystring_history = {"q":"San Diego","dt":str(self.starting_iso),"lang":"en", 'end_dt': str(self.end_date_iso)}

    def day1(self):
        self.starting_date = self.current_day_iso
        self.starting_iso = self.current_day.strftime('%Y-%m-%d')
        self.end_date = self.current_day + timedelta(days = 6)
        self.end_date_iso = self.end_date.strftime('%Y-%m-%d')
        querystring_history = {"q":"San Diego","dt":str(self.starting_iso),"lang":"en", 'end_dt': str(self.end_date_iso)}

    def viewCurrentWeek(self):
        user_weather_desire = input('Do you want to see the weather? Y/N')
        if user_weather_desire.upper() == 'Y':
            self.getWeather(self)
        else:
            this_day = datetime.today()
            this_week = this_day.isocalendar()[1]
            this_year = this_day.isocalendar()[0]
            current_week_1 = list(filter(lambda x: x['_date'].isocalendar()[0] == this_year and x['_date'].isocalendar()[1] == this_week, self.innerHolidays))
            for item in current_week_1:
                item_date = datetime.strftime(item['_date'], '%Y-%m-%d')
                print(f'{item["_name"]} - {item_date}')

    def displayHolidaysinWeek(self):
        display_input = input("What week do you want to see? choose a week from 1-53. It's odd to have a 53rd week, but that's how this format is. Enter 0 for current week")
        year_input = input("What year should this be from? Choose a year within 2020 and 2024. Enter 0 again for the current year")
        
        display_input = int(display_input)
        year_input = int(year_input)

        try:

            if int(display_input) == 0 and int(year_input == 0):
                self.viewCurrentWeek(self)
        
            else:
                current_week_2 = list(filter(lambda x: x['_date'].isocalendar()[0] == year_input and x['_date'].isocalendar()[1] == display_input, 
                self.innerHolidays))
                list(map(lambda x: print(x['_name'], " - ", x['_date']), current_week_2))

        except:
            print('Something went wrong. Are the dates correct?')

full_list = HolidayList


#### Full Run

In [12]:
introduction_file = open('Introduction.txt')
introduction_read = introduction_file.read()

def user_desire():
    global user_run_input
    user_run_input = int(input(introduction_read))

def main():

    global save_value

    global user_run_input

    try:
        while (user_run_input) != 11:
            
            user_desire()

            if (user_run_input) == 1:
                holiday_obj = input('Give a holiday in the following format: holiday_name, yyyy-mm-dd')
                holiday_name = holiday_obj.split(", ")[0]
                holiday_date = holiday_obj.split(", ")[1]
                try:
                    full_list.addHoliday(full_list, Holiday(holiday_name, holiday_date))
                    print("Holiday successfully added!")
                    save_value = 0
                except:
                    print("Did you input everything correctly? Is the date right?")
                print()
            
            elif(user_run_input) == 2:
                holiday_remove = input("Please remove a holiday; use the following format: holiday_name, yyyy-mm-dd")
                holiday_name = holiday_remove.split(", ")[0]
                holiday_date = holiday_remove.split(", ")[1]
                try:
                    full_list.removeHoliday(full_list, holiday_name, holiday_date)

                    print('Holiday successfully removed.')
                    
                    save_value = 0
                except:
                    print("Did you input everything correctly? Is the date right?")
                print()

            elif(user_run_input) == 3:
                print(len(full_list.innerHolidays))
            
            elif(user_run_input) == 4:
                for item in full_list.innerHolidays:
                    print(item['_name'], datetime.date(item['_date']))

            elif(user_run_input) == 5:
                full_list.scrapeHolidays(full_list)
                save_value = 0
                print("Web scraping complete.")
                print()
            
            elif(user_run_input) == 6:
                try:
                    full_list.getWeather(full_list)
                    save_value = 0
                except:
                    print("This isn't working. Perhaps the API isn't very friendly today.")
                print()
            
            elif(user_run_input) == 7:
                try:
                    file_desired = input("What file would you like to use? Please include the .json extension")
                    full_list.read_json(full_list, file_desired)
                    print("Json successfully read in")
                except:
                    print("Something went wrong. Did you use the right file name conventions?")
                    save_value = 0

            elif(user_run_input) == 8:
                try:
                    output_file = open("Holiday_List_Output.json", "w")
                    json.dump({'holidays':full_list.innerHolidays}, output_file, default = str, indent = 2)
                    save_value = 1
                    print("Successfully saved!")
                except:
                    print("Something went wrong. Try again!")
                    print()

            elif(user_run_input) == 9:
                full_list.viewCurrentWeek(full_list)
                print()
            
            elif(user_run_input) == 10:
                full_list.displayHolidaysinWeek(full_list)
                print()

        if user_run_input == 11:
            if save_value == 0:
                leave_value = input("Are you sure you want to exit? You have unsaved changes (Y/N)")
                if leave_value.upper() == 'Y':
                    return
                elif leave_value.upper() == 'N':
                    user_run_input = 12
                    main()
            elif save_value == 1:
                leave_value = input("Are you sure you want to exit? (Y/N)")
                if leave_value.upper() == 'Y':
                    return
                elif leave_value.upper() == 'N':
                    user_run_input = 12
                    main()

    except:
        print("Something went wrong")
        print()
        main()

save_value = 0
user_run_input = 13
main()
    